# Exercise Sheet 1: Python Basics

This first  exercise sheet tests the basic functionalities of the Python programming language in the context of a simple prediction task. We consider the problem of predicting health risk of subjects from personal data and habits. We first use for this task a decision tree

![](tree.png)

adapted from the webpage http://www.refactorthis.net/post/2013/04/10/Machine-Learning-tutorial-How-to-create-a-decision-tree-in-RapidMiner-using-the-Titanic-passenger-data-set.aspx. For this exercise sheet, you are required to use only pure Python, and to not import any module, including numpy. In exercise sheet 2, the nearest neighbor part of this exercise sheet will be revisited with numpy.

## Classifying a single instance (15 P)

* Create a function that takes as input a tuple containing values for attributes (smoker,age,diet), and computes the output of the decision tree.
* Test your function on the tuple `('yes',31,'good')`,

In [239]:
#calcRisk takes a tuple as input and return a new tuple, now including a calculated risk
def calcRisk(personData):
    
    #calculate the risk acccording to the decision tree
    if personData[0] == 'yes':
        if personData[1] >= 29.5:
            risk = 'more'
        elif personData[1] <= 29.5:
            risk = 'less'
        #since the 29.5 is not included in the tree it will be undefined
        else:
            risk = 'unknown'
    elif personData[0] == 'no':
        if personData[2] == 'good':
            risk = 'less'
        elif personData[2] == 'poor':
            risk = 'more'
        else:
            risk = 'unknown'
    else:
        risk = 'unknown'

    # create a new tuple in the following format: ((’yes’, 31, ’good’), ’more’)
    riskData = tuple(personData), risk
    
    return riskData

#print
calcRisk(('yes',31,'good'))

(('yes', 31, 'good'), 'more')

## Reading a dataset from a text file (10 P)

The file `health-test.txt` contains several fictious records of personal data and habits.

* Read the file automatically using the methods introduced during the lecture.
* Represent the dataset as a list of tuples.

In [242]:
def readPatientList(filename):
    f = open(filename,'r')

    #create list
    dataset = []
    lineData = ()


    #create a list, including all the lines of the file
    for line in f:
        #read a line and save it in the tuple 'lineData'
        lineData = str.split(line[:-1],',')

        #convert the second entry to int
        lineData[1] = int(lineData[1])

        #add the tuple to the list
        dataset.append(tuple(lineData))

    #print dataset    
    return dataset

dataset = readPatientList('health-test.txt')

## Applying the decision tree to the dataset (15 P)

* Apply the decision tree to all points in the dataset, and compute the percentage of them that are classified as "more risk".

In [243]:
#create a counter for patients with 'more' risk
riskyPatientCounter = 0

#for each patient
for item in dataset:
    #calc risk and count if 'more' risky
    riskList = calcRisk(item)
    if riskList[1] == 'more':
        riskyPatientCounter = riskyPatientCounter + 1

#print the percentage of more risky patients
print(riskyPatientCounter / float(len(dataset)))

0.375


## Learning from examples (10 P)

Suppose that instead of relying on a fixed decision tree, we would like to use a data-driven approach where data points are classified based on a set of training observations manually labeled by experts. Such labeled dataset is available in the file `health-train.txt`. The first three columns have the same meaning than for `health-test.txt`, and the last column corresponds to the labels.

* Write a procedure that reads this file and converts it into a list of pairs. The first element of each pair is a triplet of attributes, and the second element is the label.

In [244]:
#read patient list from file
dataSetTrainRaw = readPatientList('health-train.txt')

datasetTrain = []

#format patient list
for item in dataSetTrainRaw:    
    #restructure tuple from ('yes', 54, 'good', 'less') to (('yes', 54, 'good'), 'less')
    lineData = tuple(item[:3]),item[3]
    datasetTrain.append(tuple(lineData))

# dataset
datasetTrain

[(('yes', 54, 'good'), 'less'),
 (('no', 55, 'good'), 'less'),
 (('no', 26, 'good'), 'less'),
 (('yes', 40, 'good'), 'more'),
 (('yes', 25, 'poor'), 'less'),
 (('no', 13, 'poor'), 'more'),
 (('no', 15, 'good'), 'less'),
 (('no', 50, 'poor'), 'more'),
 (('yes', 33, 'good'), 'more'),
 (('no', 35, 'good'), 'less'),
 (('no', 41, 'good'), 'less'),
 (('yes', 30, 'poor'), 'more'),
 (('no', 39, 'poor'), 'more'),
 (('no', 20, 'good'), 'less'),
 (('yes', 18, 'poor'), 'less'),
 (('yes', 55, 'good'), 'more')]

## Nearest neighbor classifier (25 P)

We consider the nearest neighbor algorithm that classifies test points following the label of the nearest neighbor in the training data. For this, we need to define a distance function between data points. We define it to be

`d(a,b) = (a[0]!=b[0])+((a[1]-b[1])/50.0)**2+(a[2]!=b[2])`

where `a` and `b` are two tuples corrsponding to the attributes of two data points.

* Write a function that retrieves for a test point the nearest neighbor in the training set, and classifies the test point accordingly.
* Test your function on the tuple `('yes',31,'good')`

In [247]:
#gets tuple and return nearest tuple from dataset
def calcNearestNeighbour(personData):
    
    distances = []
    
    #calc distance for each dataset entry (collect in list)
    for item in datasetTrain:
        distances = distances + [(personData[0]!=item[0][0])+((personData[1]-item[0][1])/50.0)**2+(personData[2]!=item[0][2])]

    #return the dataset entry with the smallest calculated distance
    return(datasetTrain[distances.index(min(distances))])

calcNearestNeighbour(('yes',31,'good'))

(('yes', 33, 'good'), 'more')

* Apply both the decision tree and nearest neighbor classifiers on the test set, and find the data point(s) for which the two classifiers disagree, and with which probability it happens.

In [249]:
datsetTest = readPatientList('health-test.txt')

unequalsCounter = 0
unequalRisks = []

for item in datasetTest:
    #if the risk entries in the train and the tree datasets differ for a patient
    if calcNearestNeighbour(item)[1] != calcRisk(item)[1]:
        #save the current patient's data and add 1 to the counter
        unequalRisks.append(item)
        unequalsCounter = EqualsCounter + 1
        
#print the patient's data and the percentage of failure
unequalRisks, unequalsCounter/float(len(datasetTest))


([('yes', 51, 'good')], 0.25)

One problem of simple nearest neighbors is that one needs to compare the point to predict to all data points in the training set. This can be slow for datasets of thousands of points or more. Alternatively, some classifiers train a model first, and then use it to classify the data.

## Nearest mean classifier (25 P)

We consider one such trainable model, which operates in two steps:

(1) Compute the average point for each class, (2) classify new points to be of the class whose average point is nearest to the point to predict.

For this classifier, we convert the attributes smoker and diet to real values (for smoker: yes=1.0 and no=0.0, and for diet: good=0.0 and poor=1.0), and use the modified distance function:

`d(a,b) = (a[0]-b[0])**2+((a[1]-b[1])/50.0)**2+(a[2]-b[2])**2`

We adopt an object-oriented approach for building this classifier.

* Implement the methods `train` and `predict` of the class `NearestMeanClassifier`.

In [ ]:
class NearestMeanClassifier:
    
    # Training method that takes as input a dataset
    # and produces two internal vectors corresponding
    # to the mean of each class.
    def train(self,dataset):
        ### Replace by your own code
        pass
        ###
    
    # Prediction method that takes as input a new data
    # point and predicts it to belong to the class with
    # nearest mean.
    def predict(self,x):
        ### Replace by your own code
        pass
        ###

* Build an object of class `NearestMeanClassifier`, train it on the training data, and print the mean vector for each class.

In [ ]:
### Replace by your own code
import solutions
solutions.exercise6a()
###

* Predict the test data using the nearest mean classifier and print all test examples for which all three classifiers (decision tree, nearest neighbor and nearest mean) agree.

In [ ]:
### Replace by your own code
import solutions
solutions.exercise6b()
###